In [1]:
# FIXME:
from sc3.all import *

import random
import time
import numpy as np
from math import inf, log, sin, ceil
from functools import partial

from instruments import *
from fxs import *
from rhythms import *
from sc3_stuff import *


INFO:sc3.synth.server:setting global variable 's' to 'localhost'


In [2]:
restart()

INFO:sc3.synth.server:booting server 'localhost' on address 127.0.0.1:57110
INFO:SERVER.stdout:Number of Devices: 4
INFO:SERVER.stdout:   0 : "External Headphones"
INFO:SERVER.stdout:   1 : "MacBook Pro Microphone"
INFO:SERVER.stdout:   2 : "MacBook Pro Speakers"
INFO:SERVER.stdout:   3 : "ZoomAudioD"
INFO:SERVER.stdout:
INFO:SERVER.stdout:"MacBook Pro Microphone" Input Device
INFO:SERVER.stdout:   Streams: 1
INFO:SERVER.stdout:      0  channels 1
INFO:SERVER.stdout:
INFO:SERVER.stdout:"External Headphones" Output Device
INFO:SERVER.stdout:   Streams: 1
INFO:SERVER.stdout:      0  channels 2
INFO:SERVER.stdout:
INFO:SERVER.stdout:SC_AudioDriver: sample rate = 48000.000000, driver's block size = 512
INFO:SERVER.stdout:SuperCollider 3 server ready.
INFO:sc3.synth._serverstatus:'localhost': requested registration id
INFO:sc3.synth._serverstatus:'localhost': setting client_id to 0


In [3]:
pan_mode = random.choice([0, 1])
note_count = 0

def play_notes_from_scale(scale, notes, synth=fm_2op_sinfb, amp=1, debug=False):
    global pan_mode, note_count
    dur = 0.2
    prints = []
    for i, n in enumerate(notes):
        if n == '.':
            continue
        edo_n = scale[n % len(scale)] + edo * (n // len(scale)) # TODO: use function
        prints.append(edo_n)
        freq = base_freq * (2 ** (edo_n / edo))
        if pan_mode == 0:
            pan_value = random.uniform(-1.0, 1.0)
        else:
            pan_value = sin(note_count / 3)
        synth(0, freq, dur=dur, pan=pan_value,
              add_action='head', amp=max(0.1, 0.4 - freq / 440 / 4) * amp)
        note_count += 1
        if note_count % 200 == 0:
            pan_mode = random.choice([0, 1])
    if debug:
        print('notes:', prints)

def play_and_wait(scale, notes, synth=fm_2op_sinfb, dur=1.0):
    play_notes_from_scale(scale, notes, synth=synth)
    time.sleep(dur)

def arpeggiate_and_wait(scale, notes, synth=fm_2op_sinfb, dur=0.25):
    for n in notes:
        play_notes_from_scale(scale, [n], synth=synth)
        time.sleep(dur)

def play_seq(seq):
    dur=0.2
    for n in seq:
        if type(n) is int:
            n = [n]
        if type(n) is not list:
            time.sleep(dur)
        else:
            play_and_wait(scale, n, dur=dur)

def play_seqs(scale, seqs, synths, amps, dur=0.2):
    assert(len(seqs) == len(synths))
    max_idx = max([len(s) for s in seqs])
    for idx in range(max_idx):
        for j in range(len(seqs)):
            if len(seqs[j]) <= idx:
                continue
            n = seqs[j][idx]
            if type(n) is int:
                n = [n]
            if type(n) is not list:
                continue
            play_notes_from_scale(scale, n, synth=synths[j], amp=amps[j])
        time.sleep(dur)


In [4]:
def get_12edo(scale, edo, n):
    edo_n = scale[n % len(scale)] + edo * (n // len(scale)) # TODO: use function
    n_in_12_edo = edo_n / edo * 12
    return n_in_12_edo


def get_scale_steps(scale, edo, lo_ji, hi_ji):
    min_scale_steps = None
    max_scale_steps = None
    for i in range(1, edo):
        ratio = 2 ** (i / edo)
        if lo_ji <= ratio < hi_ji:
            if min_scale_steps is None:
                min_scale_steps = i
            max_scale_steps = i
        if hi_ji < ratio:
            break
#     print(min_scale_steps, max_scale_steps)
            
    scale_steps = []
    for step in range(1, len(scale) - 1):
        scale_with_step = list(scale[step:]) + [(n + edo) for n in scale[:step]]
        diffs = np.array(scale_with_step) - np.array(scale)
        mean_diff = np.mean(diffs)
#         print('mean', step, mean_diff)
        if min_scale_steps - 1 < mean_diff < max_scale_steps + 1:
            scale_steps.append(step)
#     print(scale_steps)
    return scale_steps

def get_semitones(scale, edo):
    return get_scale_steps(scale, edo, 24 / 25, 13 / 12)

def get_thirds(scale, edo):
    return get_scale_steps(scale, edo, 15 / 13, 17 / 13)

def get_fourths(scale, edo):
    return get_scale_steps(scale, edo, 14 / 11, 23 / 16)


def check_mos(edo, collection, min_step_in_cents=60, max_step_in_cents=350):
    collection_list = list(collection)
    collection_list.sort()
    collection_list.append(edo)
    diffs = set()
    for i in range(1, len(collection_list)):
        diff = collection_list[i] - collection_list[i - 1]
        diff_in_cents = diff * 1200 / edo
        if diff_in_cents < min_step_in_cents or diff_in_cents > max_step_in_cents:
            return False
        diffs.add(diff)
    if len(diffs) > 2:
        return False
    semitones = get_semitones(collection_list, edo)
    semitone_count = 0
    for i in range(1, len(collection_list)):
        diff = collection_list[i] - collection_list[i - 1]
        if diff in semitones:
            semitone_count += 1
            if semitone_count >= 2:
                return False
        else:
            semitone_count = 0
    return True

    
def gen_moses(edo, min_step_in_cents=60, max_step_in_cents=350,
              min_notes=5, max_notes_frac=0.6):
    moses = set()
    for step in range(1, edo):
        collection = set()
        cur = 0
        for k in range(int(max_notes_frac * edo + 1)):
            if cur in collection:
                break
            collection.add(cur)
            cur = (cur + step) % edo
            if len(collection) < min_notes:
                continue
            if len(collection) > max_notes_frac * edo:
                break
            if k > 1 and check_mos(edo, collection,
                                   min_step_in_cents=min_step_in_cents,
                                   max_step_in_cents=max_step_in_cents):
                notes = sorted(collection)
                for shift in range(edo):
                    mos = sorted([(n + shift) % edo for n in notes])
                    moses.add(tuple(mos))
    return list(moses)

def gen_scales(edo):
    return gen_moses(edo)

In [5]:
beat_count = 4
subbeat_count = 4
bpm = 120 # TODO

base_freq = 261.6255653006

In [6]:
def find_nearest_note(scale, edo, cur, targets, prob=0.75, min_bound=None):
    residue = cur % len(scale)
    best_dist = None
    best_cand = None
    targets1 = list(set(targets))
    random.shuffle(targets1)
    for target in targets1:
        target_residue = target % len(scale)
        shifts = [-1, 0, 1]
        random.shuffle(shifts)
        for k in shifts:
            candidate = target - target_residue + residue + k * len(scale)
            if min_bound is not None and get_12edo(scale, edo, candidate) < min_bound:
                continue
            # FIXME: in theory we can get None for best_cand
            cur_dist = abs(candidate - target)
            if best_dist is None or (random.random() <= prob and cur_dist < best_dist):
                best_cand = candidate
                best_dist = cur_dist
        if best_cand is None:
            print('ERROR: best cand is None!')
            best_cand = candidate
    # todo: forbid voice crossings
    return best_cand

def smooth_chords(scale, edo, chords, start=0, min_bound=None, max_bound=None, prob=0.75):
    if start is None:
        prev_chord = [0]
    else:
        if type(start) is int:
            prev_chord = [start]
        else:
            prev_chord = start
    smoothed_chords = []
    for idx, chord in enumerate(chords):
        smoothed_chord = []
        for n in chord:
            smoothed_chord.append(find_nearest_note(scale, edo, n, prev_chord, min_bound=min_bound))
        
        # trying to place the root below other notes
        if (smoothed_chord[0] != min(smoothed_chord)) and (random.random() <= prob):
            if (random.random() <= 0.5):
                if (min_bound is None) or \
                        (get_12edo(scale, edo, smoothed_chord[0] - len(scale)) >= min_bound):
                    smoothed_chord[0] -= len(scale)
            else:
                if (max_bound is None) or \
                        (get_12edo(scale, edo, max(smoothed_chord[1:]) + len(scale)) <= max_bound):
                    smoothed_chord = [p + len(scale) for p in smoothed_chord]
                    smoothed_chord[0] -= len(scale)
        if max_bound is not None and\
                (get_12edo(scale, edo, max(smoothed_chord)) > max_bound):
            if (get_12edo(scale, edo, min(smoothed_chord) - len(scale)) > min_bound):
                smoothed_chord = [p - len(scale) for p in smoothed_chord]
        # todo: check the distance between notes
        smoothed_chords.append(smoothed_chord)
        prev_chord = smoothed_chord
    return smoothed_chords

def gen_chord_progression(scale, edo, tonic,
                          root_step=4, chord_step=2, chord_len=4, scheme=0,
                          prev_chord=None, min_bound=None, max_bound=None):
    roots = []
    root = tonic
    for i in range(beat_count):
        roots.append(root)
        root += root_step
    if scheme == 1:
        roots.reverse()
        roots = [roots[-1]] + roots[:-1]
    elif scheme == 2:
        roots.reverse()
    chords = []
    for root in roots:
        chord = []
        n = root
        for i in range(chord_len):
            chord.append(n)
            n += chord_step
        chords.append(chord)
    chords = smooth_chords(scale, edo, chords, start=prev_chord,
                           min_bound=min_bound, max_bound=max_bound)
    return chords
    # todo: add more schemes, e. g.:
    # i-i-ii-ii
    # i-i-i-ii
    # i-ii-i-ii
    # add slower changes, over several bars
    # ...


def gen_random_chord_progression(scale, edo, prev_chord=None, min_bound=None, max_bound=None):
    scheme = random.randint(0, 2)
    root_step = random.randint(1, len(scale) - 1)
    thirds = get_thirds(scale, edo)
    fourths = get_fourths(scale, edo)
    chord_step = random.choice(list(set(thirds + fourths)))
    chord_len = random.randint(3, 4)
#     modulate = random.choice([False, True])
    return gen_chord_progression(scale, edo, 0,
                                 root_step=root_step,
                                 chord_step=chord_step,
                                 chord_len=chord_len,
                                 scheme=scheme,
                                 prev_chord=prev_chord,
                                 min_bound=min_bound,
                                 max_bound=max_bound)
#                                  modulate=modulate)


In [7]:
def create_rhythm():
    rhythm = []
    for i in range(beat_count):
        note_count = random.randint(1, subbeat_count)
        subrhythm = euclidean(subbeat_count, note_count)
        shift = random.randint(0, subbeat_count - 1)
        subrhythm = subrhythm[shift:] + subrhythm[:shift]
        rhythm += subrhythm
    return rhythm

def create_rhythm2():
    rhythm = []
    total_count = beat_count * subbeat_count
    note_count = random.randint(beat_count, total_count)
    rhythm = euclidean(total_count, note_count)
    shift = random.randint(0, subbeat_count - 1)
    rhythm = rhythm[shift:] + rhythm[:shift]
    return rhythm

def gen_random_percs(mode):
    rhythm = create_rhythm2()
    total_count = beat_count * subbeat_count
    assert(len(rhythm) == total_count)
    percs = []
    rep = 0
    for i in range(beat_count * subbeat_count):
        if rhythm[i] == 0:
            percs.append('.')
        else:
            percs.append(0)
    if mode == 'snare':
        for i in range(beat_count):
            for j in range(subbeat_count):
                note_idx = i * subbeat_count + j
                if i % 2 == 0:
                    percs[note_idx] = '.'
    return percs
    
def gen_melody(scale, edo, chords, start=0, max_jump=None, max_rep=2, min_bound=None, max_bound=None,
               dont_dup=False):
    if max_jump is None:
        max_jump = random.choice([2, 3])
    rhythm = create_rhythm()
    total_count = beat_count * subbeat_count
    assert(len(rhythm) == total_count)
    assert(len(chords) == beat_count)
    note_idx = -1
    direction = 1
    last_note = start
    melody = []
    rep = 0
    for i in range(beat_count):
        had_prev_jump = False
        prev_jump = None
        for j in range(subbeat_count):
            if j == 0:
                available_notes = [p % len(scale) for p in chords[i]]
                if random.random() < 0.5:
                    direction *= -1
                if get_12edo(scale, edo, last_note) >= max_bound:
                    direction = -1
                if get_12edo(scale, edo, last_note) <= min_bound:
                    direction = 1
            else:
                available_notes = list(range(len(scale)))
            note_idx += 1
            if rhythm[note_idx] == 0:
                melody.append(['.'])
                continue
            if had_prev_jump and random.random() < 0.75:
                new_note = last_note + prev_jump
                last_note = new_note
            else:
                candidates = []
                jump = -1
                while len(candidates) <= 1 or jump <= max_jump:
                    jump += 1
                    new_note = last_note + jump * direction
                    if new_note % len(scale) in available_notes:
                        candidates.append(new_note)
                while True:
                    new_note = random.choice(candidates)
                    if new_note == last_note:
                        rep += 1
                        if rep > max_rep:
                            continue
                    else:
                        rep = 0
                        had_prev_jump = True
                        prev_jump = new_note - last_note
                        last_note = new_note
                        break
            melody.append([last_note])
    duplicate = False
    if not dont_dup and random.random() < 0.45:
        duplicate = True
    if duplicate:
        thirds = get_thirds(scale, edo)
        fourths = get_fourths(scale, edo)
        duplicate_step = random.choice(list(set(thirds + fourths)))
        duplicate_delay = random.choice([0, -1, 1])
        for idx in range(len(melody)):
            if melody[idx][0] == '.':
                continue
            if 0 <= idx + duplicate_delay < len(melody):
                melody[idx + duplicate_delay].append(melody[idx][0] + duplicate_step)
    return melody, last_note, duplicate


def gen_chord_seq(scale, edo, chords):
    assert(len(chords) == beat_count)
    chord_seq = []
    schemes = [0, 1, 2, 3, 4]
    random.shuffle(schemes)
    schemes = schemes[:2]
    for i in range(beat_count):
        chord = chords[i]
        scheme = random.choice(schemes)
        if scheme == 0:
            for j in range(subbeat_count):
                if j % 2 == 1:
                    chord_seq.append('.')
                else:
                    chord_seq.append([chord[0], random.choice(chord[1:])])
        elif scheme == 1:
            cur_seq = []
            for j in range(subbeat_count):
                cur_seq.append([])
            cur_seq[0].append(chord[0])
            for j in range(1, len(chord)):
                cur_seq[random.randint(0, subbeat_count - 1)].append(chord[j])
            for j in range(subbeat_count):
                if not cur_seq[j]:
                    cur_seq[j] = '.'
            chord_seq += cur_seq
        elif scheme == 2:
            interval = [chord[0], random.choice(chord[1:])]
            for j in range(subbeat_count):
                chord_seq.append(interval[j % len(interval)])
        elif scheme == 3:
            for j in range(subbeat_count):
                chord_seq.append(chord[j % len(chord)])
        else:
            assert(scheme == 4)
            for j in range(subbeat_count):
                if j == 0:
                    shuffled_chord = chord[1:]
                    random.shuffle(shuffled_chord)
                    chord_seq.append([chord[0]] + shuffled_chord[:2])
                else:
                    chord_seq.append(random.choice(chord))
    return chord_seq


In [8]:
def convert_to_edo(scale, edo, notes):
    edo_notes = []
    for n in notes:
        edo_n = scale[n % len(scale)] + edo * (n // len(scale))
        edo_notes.append(edo_n)
    return edo_notes

def approximate_note(scale, edo, n, old_scale, old_edo):
    n_in_old_edo = convert_to_edo(old_scale, old_edo, [n])[0]
    octave = n_in_old_edo // old_edo
    n_in_edo = n_in_old_edo / old_edo * edo
    nearest = None
    best_dist = None
    for n2 in range(len(scale)):
        n2_in_octave = n2 + octave * len(scale)
        n2_in_octave_in_edo = convert_to_edo(scale, edo, [n2_in_octave])[0]
        cur_dist = abs(n2_in_octave_in_edo - n_in_edo)
        if nearest is None or cur_dist < best_dist:
            nearest = n2_in_octave
            best_dist = cur_dist
    return nearest


def modulate_and_recalc(scale, edo, chord, root):
    chord_in_edo = []
    for n in chord:
        chord_in_edo.append(scale[n % len(scale)])
    # FIXME: don't know why but this alternative fails:
    # chord_in_edo = convert_to_edo(scale, edo, chord)
    # chord_in_edo = [n % edo for n in chord_in_edo]
    # probably because of negative values, and using % operator
    available_scales = []
    for cand_scale in all_scales:
        is_good = True
        for n in chord_in_edo:
            if n not in cand_scale:
                is_good = False
                break
        if is_good:
            available_scales.append(cand_scale)
    if scale not in available_scales:
        print('ERROR: chord mismatches scale', scale, chord_in_edo)
    assert(scale in available_scales)
    new_scale = random.choice(available_scales)
    if new_scale == scale:
        new_root = (root + random.randint(1, len(scale) - 1)) % len(scale)
    else:
        new_root = random.randint(0, len(scale) - 1)
    return new_scale, new_root


In [9]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def print_cut(*inputs):
    wrap_len=70
    s = ''
    for inp in inputs:
        s += str(inp) + ' '
    s = s.split(' ')
    batch = ''
    for part in s:
        if len(batch) + 1 + len(part) > wrap_len:
            print(batch + ' ...')
            batch = '... ' + part
        else:
            batch += ' ' + part
    if batch:
        print(batch)


def get_random_fm(edo, synth=fm_2op_sinfb):
    return partial(synth,
                   m_ratio=random.uniform(1.0, 1.0),
                   c_ratio=random.uniform(1.0, 1.0),
                   index=random.choice([0, 1, 2, 3]),
                   i_scale=random.randint(1, 5),
                   decay=0.3)


In [10]:
printmd('# HI!')

dur = random.uniform(0.12, 0.3)
note_count = 0
beat_count = random.randint(2, 5)
subbeat_count = random.randint(2, 5)

# all_edos = [9, 12, 15, 17, 19, 22, 23, 26, 27, 31]
all_edos = list(range(9, 31+1))
edo = random.choice(all_edos)
all_scales = gen_scales(edo)
scale = random.choice(all_scales)

prev_chord = [-len(scale)]
root = random.randint(0, len(scale) - 1)
start_note = root
start_note2 = root

def get_random_bass():
    return random.choice([(chiptune_varsaw, 0.9),
                          (chiptune_triangle, 0.8)])

def get_random_solo():
    return random.choice([(chiptune_triangle, 1),
                          #(pluck, 0.5),
                          (kalimba, 5),
                          (fm_2op_pulse, 0.2), (fm_2op_saw, 0.2)])

def get_random_solo2():
    return random.choice([(chiptune_triangle, 0.5),
                          (kalimba, 3),
                          #(pluck, 0.2), 
                          (fm_2op_pulse, 0.1), (fm_2op_saw, 0.1)])


bass, bass_amp = get_random_bass()
solo, solo_amp = get_random_solo()
solo2, solo2_amp = get_random_solo2()

counter = 0
printmd('# edo: ' + str(edo))
print('scale', scale)
start = time.perf_counter()
last_edo_change = time.perf_counter()
edo_change_period = random.randint(60, 120)
last_instrument_switch = time.perf_counter()
last_tempo_switch = time.perf_counter()
while time.perf_counter() - start < 20*60:
    kicks = gen_random_percs(mode='kick')
    hihats = gen_random_percs(mode='hihat')
    snares = gen_random_percs(mode='snare')
    chords = gen_random_chord_progression(scale, edo, prev_chord=prev_chord, min_bound=-20, max_bound=-7)
    prev_chord = chords[-1]
    melody, last_note, is_dup = gen_melody(scale, edo, chords, start=start_note, min_bound=-4, max_bound=20)
    start_note = last_note
    melody2, last_note2, _ = gen_melody(scale, edo, chords, start=start_note2, min_bound=-16, max_bound=0,
                                        dont_dup=is_dup)
    start_note2 = last_note2
    chord_seq = gen_chord_seq(scale, edo, chords)
    print_cut('chords', chord_seq)
    print_cut('melody', melody)
    print_cut('melody2', melody2)
    switch_to_new_edo = False
    if time.perf_counter() - last_edo_change > edo_change_period:
        switch_to_new_edo = True
    modulate = False
    if not switch_to_new_edo:
        modulate = random.random() < 0.75
    reps = 2
    if not modulate and not switch_to_new_edo:
        reps = random.randint(2, 5)
    switch_instruments = (random.random() < 0.2) and (time.perf_counter() - last_instrument_switch > 10)
    if time.perf_counter() - last_instrument_switch > 30:
        switch_instruments = True
    switch_instruments_rep = random.randint(0, reps - 1)
    for j in range(reps):
        counter += 1
        mute_count = 0
        max_mute_count = 2
        cur_bass_amp = bass_amp
        cur_solo_amp = solo_amp
        cur_solo2_amp = solo2_amp
        if mute_count < max_mute_count:
            mute_bass = random.random() < 0.2
            if mute_bass:
                cur_bass_amp = 0
                mute_count += 1
        if mute_count < max_mute_count:
            mute_solo2 = random.random() < 0.2
            if mute_solo2:
                cur_solo2_amp = 0
                mute_count += 1
        if mute_count < max_mute_count:
            mute_solo = random.random() < 0.2
            if mute_solo:
                cur_solo_amp = 0
                mute_count += 1
        play_seqs(scale,
                  [kicks, hihats, snares, chord_seq, melody, melody2],
                  [kick, hihat, snare, bass, solo, solo2],
                  [random.uniform(0.7, 1.0), random.uniform(0.2, 0.4), random.uniform(0.2, 0.5),
                   cur_bass_amp, cur_solo_amp, cur_solo2_amp], dur=dur)
        if switch_instruments and j == switch_instruments_rep:
            last_instrument_switch = time.perf_counter()
            print('Changing instruments🎹🎸🎷')
            bass, bass_amp = get_random_bass()
            solo, solo_amp = get_random_solo()
            solo2, solo2_amp = get_random_solo2()
            switch_instruments = False
    switch_tempo = (not switch_instruments) and (random.random() < 0.2) and (time.perf_counter() - last_instrument_switch > 5)
    if time.perf_counter() - last_instrument_switch > 30:
        switch_tempo = True
    if switch_tempo:
        print('Changing tempo🥁')
        dur = random.uniform(0.12, 0.3)
        beat_count = random.randint(2, 5)
        subbeat_count = random.randint(2, 5)
    if modulate:
        print('modulating')
        new_edo = edo
        new_scale, new_root = modulate_and_recalc(scale, edo, chords[-1], root)
    elif switch_to_new_edo:
        edo_change_period = random.randint(60, 120)
        last_edo_change = time.perf_counter()
        new_edo = edo
        if len(all_edos) > 1:
            while new_edo == edo:
                new_edo = random.choice(all_edos)
        all_scales = gen_scales(new_edo)
        new_scale = random.choice(all_scales)
        new_root = random.randint(0, len(new_scale) - 1)
        printmd('# New EDO: ' + str(edo) + ' -> ' + str(new_edo))
    if modulate or switch_to_new_edo:
        start_note = approximate_note(new_scale, new_edo, start_note, scale, edo)
        start_note2 = approximate_note(new_scale, new_edo, start_note2, scale, edo)
        prev_chord_modulated = []
        for n in prev_chord:
            prev_chord_modulated.append(approximate_note(new_scale, edo, n, scale, edo))
        prev_chord = prev_chord_modulated
        edo = new_edo
        scale = new_scale
        root = new_root
        print('edo', edo)
        print('scale', scale)
printmd('# FIN!')

# HI!

# edo: 14

scale (1, 3, 5, 7, 11)
 chords [[-6, -3], [-5, -4], '.', [-4, -6], [-3, -2], '.', -7, -1, -5, ...
... [-5, 1, -8, -7], '.', '.'] 
 melody [[7], [10], ['.'], [9], [8], [7], ['.'], [9], ['.'], ['.'], ...
... [7], [5]] 
 melody2 [[1], [-2], ['.'], [-1], [0], ['.'], [-1], [-2], [-5], [-7], ...
... [-9], [-11]] 


KeyboardInterrupt: 